In [3]:
import psycopg2
import requests
import json
import pymysql

In [5]:
host = "gameexplorer.cal8aajyevac.ap-southeast-2.rds.amazonaws.com"
user = 'admin'
password = "wuzhenyu"
database = 'GameExplorer'

In [11]:
def fetch_game():
    
    host = "gameexplorer.cal8aajyevac.ap-southeast-2.rds.amazonaws.com"
    user = 'admin'
    password = "wuzhenyu"
    database = 'GameExplorer'
    connection = pymysql.connect(host=host, user=user, password=password, database=database)
    cursor = connection.cursor()
    response = requests.get('https://api.rawg.io/api/games?key=56d8217442f84e2398c806076bdff38d')
    json_data = response.json()
    game_num = int(json_data['count'])
    #Page number
    page_num = int(game_num/20)
    for k in range(1,10):
        print(k)
        url = 'https://api.rawg.io/api/games?key=56d8217442f84e2398c806076bdff38d&page='+str(k)
        response = requests.get(url)
        json_data = response.json()
        game_list = json_data['results']
        for i in game_list:
            game_id= int(i['id'])
            # Check if the game with the given ID already exists in the table
            check_sql = "SELECT COUNT(*) FROM GameList WHERE id = %d" % game_id
            cursor.execute(check_sql)
            result = cursor.fetchone()
            if result[0] > 0:
                continue
            game_name = i['name'].replace("'", " ")

            image = i['background_image']
            index = image.index('media/') + len('media/')
            game_image = image[:index] + 'crop/600/400/' + image[index:]

            genres_list = i['genres']
            game_genre = ""
            for j in genres_list:
                game_genre+= j['name'] + ', '
            game_genre = game_genre[:-2]

            platform_list = i['platforms']
            game_platform = ""
            for j in platform_list:
                j = dict(j)
                temp = dict(j['platform'])
                game_platform+= temp['name'] + ', '
            game_platform = game_platform[:-2]
            
            sql = '''insert into GameList (id, name, background, genre, platform) values ('%d', '%s', '%s', '%s',"%s")''' % (game_id,game_name,game_image,game_genre, game_platform)
            cursor.execute(sql)
            connection.commit()
fetch_game()

1
2
3
4
5
6
7
8
9
